In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import html

In [2]:
def processar_arquivo(arquivo_xml):
    try:
        # Tenta analisar o arquivo XML
        tree = ET.parse(arquivo_xml)
        root = tree.getroot()

        # Busca as informações gerais
        dados_gerais = root.find("DADOS-GERAIS")
        nome_completo = html.unescape(dados_gerais.attrib["NOME-COMPLETO"])

        print("Informações dos trabalhos encontrados (Ano >= 2021):")
        print(f"Nome: {nome_completo}")
        
        # Busca todas as ocorrências de "TRABALHO-EM-EVENTOS" e "ARTIGO-PUBLICADO"
        trabalhos_em_eventos = root.findall(".//TRABALHO-EM-EVENTOS")
        artigos_publicados = root.findall(".//ARTIGO-PUBLICADO")
        livros_publicados = root.findall(".//LIVRO-PUBLICADO-OU-ORGANIZADO")
        capitulos_publicados = root.findall(".//CAPITULO-DE-LIVRO-PUBLICADO")
        texto_jornais = root.findall(".//TEXTO-EM-JORNAL-OU-REVISTA")
        Demais_producao = root.findall(".//OUTRA-PRODUCAO-BIBLIOGRAFICA")
        prefacios = root.findall(".//PREFACIO-POSFACIO")
        traducaos = root.findall(".//TRADUCAO")
        
        
        resultados_trabalho = []
        resultados_artigo = []
        resultados_livros = []
        resultados_capitulos = []
        resultados_jornais = []
        resultados_demais = []
        resultados_prefacio = []
        resultados_traducao = []


        ###################################################################### OS FOR ################################################################



        
    ############################################## Processa os trabalhos em eventos

        for trabalho_evento in trabalhos_em_eventos:
            dados_basicos = trabalho_evento.find("DADOS-BASICOS-DO-TRABALHO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO-DO-TRABALHO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_trabalho = html.unescape(dados_basicos.attrib.get("TITULO-DO-TRABALHO", ""))
                    pais_evento = html.unescape(dados_basicos.attrib.get("PAIS-DO-EVENTO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    
                    detalhamento = trabalho_evento.find("DETALHAMENTO-DO-TRABALHO")
                    titulo_anais = html.unescape(detalhamento.attrib.get("TITULO-DOS-ANAIS-OU-PROCEEDINGS", "")) if detalhamento is not None else ""
                    
                    autores = trabalho_evento.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]
                    
                    Palavra_chaves = trabalho_evento.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    # Extração das áreas do conhecimento
                    areas_do_conhecimento = trabalho_evento.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = trabalho_evento.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]

                    Setores_atividade = trabalho_evento.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)
                    
                    resultados_trabalho.append((ano, titulo_trabalho, pais_evento, meio_divulgacao, titulo_anais, Nome_Citacao, nomes_autores, chaves, areas,setores,descri_adc, nome_completo))


    ########################################################## Processa os artigos publicados

        for artigo in artigos_publicados:
            dados_basicos = artigo.find("DADOS-BASICOS-DO-ARTIGO")
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO-DO-ARTIGO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_artigo = html.unescape(dados_basicos.attrib.get("TITULO-DO-ARTIGO", ""))
                    home_page = html.unescape(dados_basicos.attrib.get("HOME-PAGE-DO-TRABALHO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    
                    detalhamento = artigo.find("DETALHAMENTO-DO-ARTIGO")
                    titulo_revista = html.unescape(detalhamento.attrib.get("TITULO-DO-PERIODICO-OU-REVISTA", "")) if detalhamento is not None else ""

                    autores = artigo.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = artigo.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = artigo.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = artigo.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]
                        
                    Setores_atividade = artigo.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)
                    
                    resultados_artigo.append((ano, titulo_artigo, home_page, meio_divulgacao, titulo_revista, Nome_Citacao, nomes_autores, chaves, areas, setores, descri_adc,nome_completo))


    ########################################################## Processa os Livros

        for livro in livros_publicados:
            dados_basicos = livro.find("DADOS-BASICOS-DO-LIVRO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_livro = html.unescape(dados_basicos.attrib.get("TITULO-DO-LIVRO", ""))
                    tipo = html.unescape(dados_basicos.attrib.get("TIPO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))
                    idioma = html.unescape(dados_basicos.attrib.get("IDIOMA", ""))

                    detalhamento = livro.find("DETALHAMENTO-DO-LIVRO")
                    if detalhamento is not None:
                        volumes = html.unescape(detalhamento.attrib.get("NUMERO-DE-VOLUMES", ""))
                        paginas = html.unescape(detalhamento.attrib.get("NUMERO-DE-PAGINAS", ""))
                        isbn = html.unescape(detalhamento.attrib.get("ISBN", ""))
                        cidade = html.unescape(detalhamento.attrib.get("CIDADE-DA-EDITORA", ""))
                        edição = html.unescape(detalhamento.attrib.get("NUMERO-DA-EDICAO-REVISAO", ""))
                        Nome_editora = html.unescape(detalhamento.attrib.get('NOME-DA-EDITORA', ""))
                    else:
                        volumes = ""
                        paginas = ""
                        isbn = ""
                        cidade = ""
                        edição = ""
                        Nome_editora = ""

                    autores = livro.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = livro.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = livro.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None :
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = livro.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]
                    
                    Setores_atividade = livro.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_livros.append((ano, titulo_livro, tipo, meio_divulgacao, pais, idioma, volumes, paginas, isbn, cidade, edição, Nome_editora, Nome_Citacao, nomes_autores, chaves, areas, setores,descri_adc, nome_completo))


    ########################################################## Processa os capitulos de livros

        for capitulo in capitulos_publicados:
            dados_basicos = capitulo.find("DADOS-BASICOS-DO-CAPITULO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_capitulo = html.unescape(dados_basicos.attrib.get("TITULO-DO-CAPITULO-DO-LIVRO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))
                    idioma = html.unescape(dados_basicos.attrib.get("IDIOMA", ""))

                    detalhamento = capitulo.find("DETALHAMENTO-DO-CAPITULO")
                    if detalhamento is not None:
                        titulo_livro = html.unescape(detalhamento.attrib.get("TITULO-DO-LIVRO", ""))
                        paginas = html.unescape(detalhamento.attrib.get("PAGINA-INICIAL", "") + '-' + detalhamento.attrib.get("PAGINA-FINAL", ""))
                        cidade = html.unescape(detalhamento.attrib.get("CIDADE-DA-EDITORA", ""))
                        edição = html.unescape(detalhamento.attrib.get("NUMERO-DA-EDICAO-REVISAO", ""))
                        Nome_editora = html.unescape(detalhamento.attrib.get('NOME-DA-EDITORA', ""))
                    else:
                        titulo_livro = ""
                        paginas = ""
                        cidade = ""
                        edição = ""
                        Nome_editora = ""

                    autores = capitulo.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = capitulo.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = capitulo.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = capitulo.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]
                    
                    Setores_atividade = capitulo.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_capitulos.append((ano, titulo_capitulo, titulo_livro, meio_divulgacao, pais, idioma, paginas, cidade, edição, Nome_editora, Nome_Citacao, nomes_autores, chaves, areas, setores,descri_adc,nome_completo))


    ########################################################## Processa os Textos de jornais

        for texto in texto_jornais:
            dados_basicos = texto.find("DADOS-BASICOS-DO-TEXTO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO-DO-TEXTO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_texto = html.unescape(dados_basicos.attrib.get("TITULO-DO-TEXTO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    natureza = html.unescape(dados_basicos.attrib.get("NATUREZA", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))

                    detalhamento = texto.find("DETALHAMENTO-DO-TEXTO")
                    if detalhamento is not None:
                        titulo_jornal = html.unescape(detalhamento.attrib.get("TITULO-DO-JORNAL-OU-REVISTA", ""))
                        issn = html.unescape(detalhamento.attrib.get("ISSN", ""))
                    else:
                        titulo_jornal = ""
                        issn = ""

                    autores = texto.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = texto.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = texto.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = texto.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]

                    Setores_atividade = texto.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_jornais.append((ano, titulo_texto, meio_divulgacao, natureza, pais, titulo_jornal, issn, Nome_Citacao, nomes_autores,texto, chaves, areas,setores,descri_adc, nome_completo))

        ########################################################## Processa as demais produções 

        for demais in Demais_producao:
            dados_basicos = demais.find("DADOS-BASICOS-DE-OUTRA-PRODUCAO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_demais = html.unescape(dados_basicos.attrib.get("TITULO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    home_page = html.unescape(dados_basicos.attrib.get("HOME-PAGE-DO-TRABALHO", ""))
                    natureza = html.unescape(dados_basicos.attrib.get("NATUREZA", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))

                    detalhamento = demais.find("DETALHAMENTO-DE-OUTRA-PRODUCAO")
                    if detalhamento is not None:
                        Nome_editora = html.unescape(detalhamento.attrib.get("EDITORA", ""))
                        Cidade_editora = html.unescape(detalhamento.attrib.get("CIDADE-DA-EDITORA", ""))
                        issn_isbn = html.unescape(detalhamento.attrib.get("ISSN-ISBN", ""))
                    else:
                        Nome_editora = ""
                        Cidade_editora = ""
                        issn_isbn = ""

                    autores = demais.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = demais.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = demais.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = demais.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]

                    Setores_atividade = demais.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_demais.append((ano, titulo_demais, meio_divulgacao, home_page, natureza, pais, Nome_editora, Cidade_editora, Nome_Citacao, nomes_autores, issn_isbn,
                                            chaves, areas, setores,descri_adc, nome_completo))   
                    
                     
                    

    ########################################################## Processa as demais produções (Prefacio)

        for prefacio in prefacios:
            dados_basicos = prefacio.find("DADOS-BASICOS-DO-PREFACIO-POSFACIO")
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_prefacio = html.unescape(dados_basicos.attrib.get("TITULO", ""))
                    tipo = html.unescape(dados_basicos.attrib.get("TIPO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    home_page = html.unescape(dados_basicos.attrib.get("HOME-PAGE-DO-TRABALHO", ""))
                    natureza = html.unescape(dados_basicos.attrib.get("NATUREZA", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))

                    detalhamento = prefacio.find("DETALHAMENTO-DO-PREFACIO-POSFACIO")
                    if detalhamento is not None:
                        Nome_editora = html.unescape(detalhamento.attrib.get("EDITORA-DO-PREFACIO-POSFACIO", ""))
                        Nome_autor_publicacao = html.unescape(detalhamento.attrib.get("NOME-DO-AUTOR-DA-PUBLICACAO", ""))
                        titulo_publi = html.unescape(detalhamento.attrib.get("TITULO-DA-PUBLICACAO", ""))
                        Cidade_editora = html.unescape(detalhamento.attrib.get("CIDADE-DA-EDITORA", ""))
                        issn_isbn = html.unescape(detalhamento.attrib.get("ISSN-ISBN", ""))
                        numero_edicao = html.unescape(detalhamento.attrib.get("NUMERO-DA-EDICAO-REVISAO", ""))
                        volume = html.unescape(detalhamento.attrib.get("VOLUME", ""))
                        serie = html.unescape(detalhamento.attrib.get("SERIE", ""))
                        fasciculo = html.unescape(detalhamento.attrib.get("FASCICULO", ""))
                    else:
                        Nome_editora = ""
                        Nome_autor_publicacao = ""
                        titulo_publi = ""
                        Cidade_editora = ""
                        issn_isbn = ""
                        numero_edicao = ""
                        volume = ""
                        serie = ""
                        fasciculo = ""

                    autores = prefacio.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]

                    Palavra_chaves = prefacio.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = prefacio.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    descri_adc = prefacio.findall("INFORMACOES-ADICIONAIS")
                    descri_adc_text = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in descri_adc]


                    Setores_atividade = prefacio.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_prefacio.append((ano, titulo_prefacio, tipo, meio_divulgacao, home_page, natureza, pais, Nome_editora, Nome_autor_publicacao, titulo_publi, Cidade_editora,
                                                issn_isbn, numero_edicao, volume, serie, fasciculo, nomes_autores, Nome_Citacao, chaves, areas,setores, descri_adc_text, nome_completo))

                    
    ########################################################## Processa as demais produções (Traducao)

        for traducao in traducaos:
            dados_basicos = traducao.find("DADOS-BASICOS-DA-TRADUCAO")     
            if dados_basicos is not None:
                ano = dados_basicos.attrib.get("ANO", None)
                if ano is not None and int(ano) >= 2021:
                    titulo_traducao = html.unescape(dados_basicos.attrib.get("TITULO", ""))
                    meio_divulgacao = html.unescape(dados_basicos.attrib.get("MEIO-DE-DIVULGACAO", ""))
                    home_page = html.unescape(dados_basicos.attrib.get("HOME-PAGE-DO-TRABALHO", ""))
                    natureza = html.unescape(dados_basicos.attrib.get("NATUREZA", ""))
                    pais = html.unescape(dados_basicos.attrib.get("PAIS-DE-PUBLICACAO", ""))
                    idioma = html.unescape(dados_basicos.attrib.get("IDIOMA", ""))

                    detalhamento = traducao.find("DETALHAMENTO-DO-TRADUCAO-POSFACIO")
                    if detalhamento is not None:
                        Nome_editora = html.unescape(detalhamento.attrib.get("EDITORA-DA-TRADUCAO", ""))
                        Nome_autor_traduzido = html.unescape(detalhamento.attrib.get("NOME-DO-AUTOR-TRADUZIDO", ""))
                        titulo_original = html.unescape(detalhamento.attrib.get("TITULO-DA-OBRA-ORIGINAL", ""))
                        Cidade_editora = html.unescape(detalhamento.attrib.get("CIDADE-DA-EDITORA", ""))
                        issn_isbn = html.unescape(detalhamento.attrib.get("ISSN-ISBN", ""))
                        idioma_original = html.unescape(detalhamento.attrib.get("IDIOMA-DA-OBRA-ORIGINAL", ""))
                        numero_paginas = html.unescape(detalhamento.attrib.get("NUMERO-DE-PAGINAS", ""))
                        numero_edicao = html.unescape(detalhamento.attrib.get("NUMERO-DA-EDICAO-REVISAO", ""))
                        volume = html.unescape(detalhamento.attrib.get("VOLUME", ""))
                        serie = html.unescape(detalhamento.attrib.get("SERIE", ""))
                        fasciculo = html.unescape(detalhamento.attrib.get("FASCICULO", ""))
                    else:
                        Nome_editora = ""
                        Cidade_editora = ""
                        Nome_autor_traduzido = ""
                        titulo_original = ""
                        issn_isbn = ""
                        idioma_original = ""
                        titulo_publi = ""
                        numero_edicao = ""
                        numero_paginas = ""
                        volume = ""
                        serie = ""
                        fasciculo = ""

                    autores = traducao.findall("AUTORES")
                    nomes_autores = [html.unescape(autor.attrib.get("NOME-COMPLETO-DO-AUTOR", "")) for autor in autores]
                    Nome_Citacao = [html.unescape(autor.attrib.get("NOME-PARA-CITACAO", "")) for autor in autores]
                    ordem_autoria = [html.unescape(autor.attrib.get("ORDEM-DE-AUTORIA", "")) for autor in autores]

                    Palavra_chaves = traducao.find("PALAVRAS-CHAVE")
                    chaves = []
                    if Palavra_chaves is not None:
                        for chave in Palavra_chaves.attrib:
                            if chave.startswith("PALAVRA-CHAVE-"):
                                valor_chave = html.unescape(Palavra_chaves.attrib[chave])
                                if valor_chave:
                                    chaves.append(valor_chave)

                    areas_do_conhecimento = traducao.find("AREAS-DO-CONHECIMENTO")
                    areas = []
                    if areas_do_conhecimento is not None:
                        for area in areas_do_conhecimento:
                            nome_grande_area = html.unescape(area.attrib.get("NOME-GRANDE-AREA-DO-CONHECIMENTO", ""))
                            nome_area = html.unescape(area.attrib.get("NOME-DA-AREA-DO-CONHECIMENTO", ""))
                            nome_sub_area = html.unescape(area.attrib.get("NOME-DA-SUB-AREA-DO-CONHECIMENTO", ""))
                            nome_especialidade = html.unescape(area.attrib.get("NOME-DA-ESPECIALIDADE", ""))
                            areas.append((nome_grande_area, nome_area, nome_sub_area, nome_especialidade))

                    infoadc = traducao.findall("INFORMACOES-ADICIONAIS")
                    descri_adc = [html.unescape(info.attrib.get("DESCRICAO-INFORMACOES-ADICIONAIS", "")) for info in infoadc]

                    Setores_atividade = traducao.find("SETORES-DE-ATIVIDADE")
                    setores = []
                    if Setores_atividade is not None:
                        for setor in Setores_atividade.attrib:
                            if setor.startswith("SETOR-DE-ATIVIDADE-"):
                                valor_Setor = html.unescape(Setores_atividade.attrib[setor])
                                if valor_Setor:
                                    setores.append(valor_Setor)

                    resultados_traducao.append((ano, titulo_traducao, meio_divulgacao, home_page, natureza, pais, idioma, Nome_editora, Nome_autor_traduzido, titulo_original, 
                                                Cidade_editora, issn_isbn, idioma_original, numero_paginas, numero_edicao, volume, serie, fasciculo, nomes_autores, Nome_Citacao, 
                                                ordem_autoria, chaves, areas,setores, descri_adc, nome_completo))


        return resultados_trabalho, resultados_artigo,resultados_livros,resultados_capitulos, resultados_jornais ,resultados_demais ,resultados_prefacio ,resultados_traducao

    except FileNotFoundError:
        print(f"O arquivo {arquivo_xml} não foi encontrado.")
    except ET.ParseError:
        print(f"Erro ao analisar o arquivo {arquivo_xml}.")
    except ValueError:
        print(f"Erro ao converter o ano para inteiro no arquivo {arquivo_xml}.")
    except Exception as e:
        print(f"Ocorreu um erro no arquivo {arquivo_xml}: {e}")
        return [], [], [], [], [], [], [], []


In [3]:
# Listas para armazenar os resultados de todos os arquivos
todos_resultados_trabalho = []
todos_resultados_artigo = []
todos_resultados_livros = []
todos_resultados_capitulos = []
todos_resultados_jornais = []
todos_resultados_demais = []
todos_resultados_prefacio = []
todos_resultados_traducao = []

# Iterar sobre todos os arquivos XML na pasta "Data"
for xml_file in os.listdir("Data"):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join("Data", xml_file)
        resultados_trabalho, resultados_artigo, resultados_livros, resultados_capitulos, resultados_jornais, resultados_demais, resultados_prefacio, resultados_traducao = processar_arquivo(xml_path)
        todos_resultados_trabalho.extend(resultados_trabalho)
        todos_resultados_artigo.extend(resultados_artigo)
        todos_resultados_livros.extend(resultados_livros)
        todos_resultados_capitulos.extend(resultados_capitulos)
        todos_resultados_jornais.extend(resultados_jornais)
        todos_resultados_demais.extend(resultados_demais)
        todos_resultados_prefacio.extend(resultados_prefacio)
        todos_resultados_traducao.extend(resultados_traducao)

# Criação dos DataFrames combinados
df_trabalho = pd.DataFrame(todos_resultados_trabalho, columns=["Ano", "Titulo do Trabalho", "País do Evento", "Meio de Divulgação", "Título dos Anais", "Nome para Citação", "Nome Completo do Autor", "Palavras-chave", "Áreas do Conhecimento", "Setores", "Informações Adicionais","Nome Completo"])
df_artigo = pd.DataFrame(todos_resultados_artigo, columns=["Ano", "Título do Artigo", "Home Page", "Meio de Divulgação", "Título do Periódico/Revista", "Nome para Citação", "Nome Completo do Autor", "Palavras-chave", "Áreas do Conhecimento", "Setores", "Informações Adicionais","Nome Completo"])
df_livros = pd.DataFrame(todos_resultados_livros, columns=["Ano", "Título do Livro", "Tipo", "Meio de Divulgação", "País", "Idioma", "Volumes", "Páginas", "ISBN", "Cidade", "Edição", "Nome da Editora", "Nome para Citação", "Nome Completo do Autor", "Palavras-chave", "Áreas do Conhecimento","Setores", "Informações Adicionais", "Nome Completo"])
df_capitulos = pd.DataFrame(todos_resultados_capitulos, columns=["Ano", "Título do Capítulo", "Título do Livro", "Meio de Divulgação", "País", "Idioma", "Páginas", "Cidade", "Edição", "Nome da Editora", "Nome para Citação", "Nome Completo do Autor", "Palavras-chave", "Áreas do Conhecimento","Setores", "Informações Adicionais", "Nome Completo"])
df_jornais = pd.DataFrame(todos_resultados_jornais, columns=["Ano", "Título do Texto", "Meio de Divulgação", "Natureza", "País de Publicação", "Título do Jornal/Revista", "ISSN", "Nome para Citação", "Nome Completo do Autor", "Palavras-chave", "Áreas do Conhecimento","Setores", "Informações Adicionais", "Nome Completo"])
df_demais = pd.DataFrame(todos_resultados_demais, columns=["Ano", "Título", "Meio de Divulgação", "Home Page", "Natureza", "País de Publicação", "Nome da Editora", "Cidade da Editora", "Nome para Citação", "Nome Completo do Autor", "ISSN/ISBN", "Palavras-chave", "Áreas do Conhecimento","Setores", "Informações Adicionais", "Nome Completo"])
df_prefacio = pd.DataFrame(todos_resultados_prefacio, columns=["Ano", "Título do Prefácio", "Tipo", "Meio de Divulgação", "Home Page", "Natureza", "País de Publicação", "Nome da Editora", "Nome do Autor da Publicação", "Título da Publicação", "Cidade da Editora", "ISSN/ISBN", "Número da Edição", "Volume", "Série", "Fascículo", "Nomes dos Autores", "Nome para Citação", "Palavras-chave", "Áreas do Conhecimento", "Setores", "Informações Adicionais", "Nome Completo"])
df_traducao = pd.DataFrame(todos_resultados_traducao, columns=["Ano", "Título", "Meio de Divulgação", "Home Page", "Natureza", "País de Publicação", "Idioma", "Nome da Editora", "Nome do Autor Traduzido", "Título Original", "Cidade da Editora", "ISSN/ISBN", "Idioma Original", "Número de Páginas", "Número da Edição", "Volume", "Série", "Fascículo", "Nomes dos Autores", "Nome para Citação", "Ordem de Autoria", "Palavras-chave", "Áreas do Conhecimento", "Setores", "Informações Adicionais", "Nome Completo"])

# Salvar os DataFrames no arquivo Excel
with pd.ExcelWriter('Producao_Bibliografica.xlsx') as writer:
    df_trabalho.to_excel(writer, sheet_name='Trabalhos em Eventos', index=False)
    df_artigo.to_excel(writer, sheet_name='Artigos Publicados', index=False)
    df_livros.to_excel(writer, sheet_name='Livros Publicados', index=False)
    df_capitulos.to_excel(writer, sheet_name='Capitulos Publicados', index=False)
    df_jornais.to_excel(writer, sheet_name='Textos em Jornais', index=False)
    df_demais.to_excel(writer, sheet_name='Demais Produções', index=False)
    df_prefacio.to_excel(writer, sheet_name='Prefacio', index=False)
    df_traducao.to_excel(writer, sheet_name='Traducao', index=False)



Informações dos trabalhos encontrados (Ano >= 2021):
Nome: Silvia Regina Liebel
Informações dos trabalhos encontrados (Ano >= 2021):
Nome: Deivid Valério Gaia
Informações dos trabalhos encontrados (Ano >= 2021):
Nome: Marta Mega de Andrade
Informações dos trabalhos encontrados (Ano >= 2021):
Nome: Joao Luis Ribeiro Fragoso
Informações dos trabalhos encontrados (Ano >= 2021):
Nome: Cláudio Costa Pinheiro
